<a href="https://colab.research.google.com/github/MS1997/Semantic-Segmentation-of-Water-Bodies/blob/master/Identify_water_bodies_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

# setting all the seeds  
SEED = 101
import os
import shutil
import random as rn
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
# tf.random.set_seed(SEED)
rn.seed(SEED)

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, concatenate, Conv2DTranspose, Input, MaxPooling2D, ReLU, BatchNormalization, Dense
from keras.optimizers import Adam
from random import sample 
from keras.models import Model

In [3]:
 # installing kaggle library to import the data directly into Colab notebook
 ! pip install -q kaggle

In [ ]:
# select file containing API key from local system
from google.colab import files
uploaded = files.upload()

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# importing the chest x-ray data set from kaggle
! kaggle datasets download -d franciscoescobar/satellite-images-of-water-bodies

In [ ]:
# unzipping the files
! unzip satellite-images-of-water-bodies.zip 

In [8]:
# creating a folder for validation set of image and masks 
os.mkdir('validation')
os.mkdir('validation/Images')
os.mkdir('validation/Images/Images')
os.mkdir('validation/Masks')
os.mkdir('validation/Masks/Masks')

# to create generator objects from directory, we will need to make another folder 
os.mkdir('Water Bodies Dataset/Images/Images')
os.mkdir('Water Bodies Dataset/Masks/Masks')

# changing the location of the Images  
for i in os.listdir('Water Bodies Dataset/Images'):
  if i.endswith('.jpg'):
    os.rename('Water Bodies Dataset/Images/'+i,'Water Bodies Dataset/Images/Images/'+i)
# changing the location of the Masks  
for i in os.listdir('Water Bodies Dataset/Masks'):
  if i.endswith('.jpg'):
    os.rename('Water Bodies Dataset/Masks/'+i,'Water Bodies Dataset/Masks/Masks/'+i)

s = sample(range(0,2841),281)

img_list = sorted(os.listdir('Water Bodies Dataset/Images/Images'))
mask_list = sorted(os.listdir('Water Bodies Dataset/Masks/Masks'))

val_image = [img_list[i] for i in s]
val_mask = [mask_list[i] for i in s]

# getting validation images and their masks  
for i in os.listdir('Water Bodies Dataset/Images/Images'):
  if i in val_image:
    os.rename('Water Bodies Dataset/Images/Images/'+i,'validation/Images/Images/'+i)
for i in os.listdir('Water Bodies Dataset/Masks/Masks'):
  if i in val_mask:
    os.rename('Water Bodies Dataset/Masks/Masks/'+i,'validation/Masks/Masks/'+i)

In [9]:
# creating a folder for test set of image and masks 
os.mkdir('test')
os.mkdir('test/Images')
os.mkdir('test/Images/Images')
os.mkdir('test/Masks')
os.mkdir('test/Masks/Masks')

s = sample(range(0,281),25)

img_list = sorted(os.listdir('validation/Images/Images'))
mask_list = sorted(os.listdir('validation/Masks/Masks'))

test_image = [img_list[i] for i in s]
test_mask = [mask_list[i] for i in s]

# getting validation images and their masks  
for i in os.listdir('validation/Images/Images/'):
  if i in test_image:
    os.rename('validation/Images/Images/'+i,'test/Images/Images/'+i)
for i in os.listdir('validation/Masks/Masks'):
  if i in test_mask:
    os.rename('validation/Masks/Masks/'+i,'test/Masks/Masks/'+i)

In [226]:
# directory to store augmented images 
os.mkdir('Water Bodies Dataset/augmented')

os.mkdir('Water Bodies Dataset/augmented/Images')
os.mkdir('Water Bodies Dataset/augmented/Masks')

In [225]:
shutil.rmtree('Water Bodies Dataset/augmented')

In [ ]:
save_here_img = 'Water Bodies Dataset/augmented/Images'
save_here_msk = 'Water Bodies Dataset/augmented/Masks'

# create two instances with the same arguments
data_gen_args = dict(rescale=1.0/255.0 ) # base case
#horizontal_flip = True, vertical_flip = True - case 2
#horizontal_flip = True, vertical_flip = True, rotation_range = 20 - case 2
#width_shift_range =0.3, height_shift_range = 0.3 - case 4
# shear_range=20, zoom_range=[0.2, 0.5] - case 5

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# using same seed and keyword arguments to the fit and flow methods
seed = 101
image_generator = image_datagen.flow_from_directory('Water Bodies Dataset/Images', class_mode=None,seed=seed, target_size=(256, 256), batch_size= 16, save_to_dir=save_here_img, save_prefix = 'aug', save_format= 'jpg')
mask_generator = mask_datagen.flow_from_directory('Water Bodies Dataset/Masks',class_mode=None, seed=seed,target_size=(256, 256), batch_size=16, color_mode='grayscale',save_to_dir= save_here_msk, save_prefix='aug', save_format= 'jpg')
# color_mode = 'grayscale' for masks since they are black and white

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)
next(train_generator) # to save the first set of augmentations so that we can see if they are correct 

# validation generator
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_image_generator = val_datagen.flow_from_directory('validation/Images',target_size=(256, 256),batch_size=16,class_mode=None, seed = seed)
val_mask_generator = val_datagen.flow_from_directory('validation/Masks',target_size=(256, 256),batch_size=16,class_mode=None, seed = seed, color_mode='grayscale')
val_generator = zip(val_image_generator, val_mask_generator)

In [ ]:
l_mask = os.listdir('Water Bodies Dataset/augmented/Masks')
len(os.listdir('Water Bodies Dataset/augmented/Masks'))

l_image = os.listdir('Water Bodies Dataset/augmented/Images')
os.listdir('Water Bodies Dataset/augmented/Images')

In [25]:
# Modelling 
# we will use keras functional API 
# encoder 
# Input 
input_img = Input(shape=(256, 256, 3))
# Encoder 
c1 = Conv2D(32, (3, 3),padding='same')(input_img)
c1 = BatchNormalization()(c1)
c1 = ReLU()(c1)
c1 = Conv2D(32, (3, 3),padding='same')(c1)
c1 = BatchNormalization()(c1)
c1 = ReLU()(c1)
p1 = MaxPooling2D((2,2))(c1)

c2 = Conv2D(64, (3, 3),padding='same')(p1)
c2 = BatchNormalization()(c2)
c2 = ReLU()(c2)
c2 = Conv2D(64, (3, 3),padding='same')(c2)
c2 = BatchNormalization()(c2)
c2 = ReLU()(c2)
p2 = MaxPooling2D((2,2))(c2)

c3 = Conv2D(128, (3, 3),padding='same')(p2)
c3 = BatchNormalization()(c3)
c3 = ReLU()(c3)
c3 = Conv2D(128, (3, 3),padding='same')(c3)
c3 = BatchNormalization()(c3)
c3 = ReLU()(c3)
p3 = MaxPooling2D((2,2))(c3)

c4 = Conv2D(256, (3, 3),padding='same')(p3)
c4 = BatchNormalization()(c4)
c4 = ReLU()(c4)
c4 = Conv2D(256, (3, 3),padding='same')(c4)
c4 = BatchNormalization()(c4)
c4 = ReLU()(c4)
p4 = MaxPooling2D((2,2))(c4)

c5 = Conv2D(512, (3, 3),padding='same')(p4)
c5 = BatchNormalization()(c5)
c5 = ReLU()(c5)
c5 = Conv2D(512, (3, 3),padding='same')(c5)
c5 = BatchNormalization()(c5)
c5 = ReLU()(c5)
p5 = MaxPooling2D((2,2))(c5)

c6 = Conv2D(1024, (3, 3),padding='same')(p5)
c6 = BatchNormalization()(c6)
c6 = ReLU()(c6)
c6 = Conv2D(1024, (3, 3),padding='same')(c6)
c6 = BatchNormalization()(c6)
c6 = ReLU()(c6)

# Decoder
t1 = Conv2DTranspose(512, (3,3),strides=(2,2),padding='same')(c6)
t1 = concatenate([t1, c5]) 
t1 = BatchNormalization()(t1)
t1 = ReLU()(t1)
t1 = Conv2D(512, (3,3),padding='same')(t1)
t1 = BatchNormalization()(t1)
t1 = ReLU()(t1)
t1 = Conv2D(512, (3,3),padding='same')(t1)
t1 = BatchNormalization()(t1)
t1 = ReLU()(t1)

t2 = Conv2DTranspose(256, (3,3),strides=(2,2),padding='same')(t1)
t2 = concatenate([t2, c4])
t2 = BatchNormalization()(t2)
t2 = ReLU()(t2)
t2 = Conv2D(256, (3,3),padding='same')(t2)
t2 = BatchNormalization()(t2)
t2 = ReLU()(t2)
t2 = Conv2D(256, (3,3),padding='same')(t2)
t2 = BatchNormalization()(t2)
t2 = ReLU()(t2)

t3 = Conv2DTranspose(128, (3,3),strides=(2,2),padding='same')(t2)
t3 = concatenate([t3, c3])
t3 = BatchNormalization()(t3)
t3 = ReLU()(t3)
t3 = Conv2D(128, (3,3),padding='same')(t3)
t3 = BatchNormalization()(t3)
t3 = ReLU()(t3)
t3 = Conv2D(128, (3,3),padding='same')(t3)
t3 = BatchNormalization()(t3)
t3 = ReLU()(t3)


t4 = Conv2DTranspose(64, (3,3),strides=(2,2),padding='same')(t3)
t4 = concatenate([t4, c2])
t4 = BatchNormalization()(t4)
t4 = ReLU()(t4)
t4 = Conv2D(64, (3,3),padding='same')(t4)
t4 = BatchNormalization()(t4)
t4 = ReLU()(t4)
t4 = Conv2D(64, (3,3),padding='same')(t4)
t4 = BatchNormalization()(t4)
t4 = ReLU()(t4)

t5 = Conv2DTranspose(32, (3,3),strides=(2,2),padding='same')(t4)
t5 = concatenate([t5, c1])
t5 = BatchNormalization()(t5)
t5 = ReLU()(t5)
t5 = Conv2D(32, (3,3),padding='same')(t5)
t5 = BatchNormalization()(t5)
t5 = ReLU()(t5)
t5 = Conv2D(32, (3,3),padding='same')(t5)
t5 = BatchNormalization()(t5)
t5 = ReLU()(t5)

y = Conv2DTranspose(1, (1,1), activation='sigmoid')(t5)


In [26]:
model_1 = Model(input_img,y)
model_2 = Model(input_img,y)
model_3 = Model(input_img,y)
model_4 = Model(input_img,y)
model_5 = Model(input_img,y)


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("/content/gdrive/My Drive/mw.h5", save_weights_only = False, period = 20) # save model weights after 20 epochs
callbacks_list = [checkpoint]

In [ ]:
model.compile(loss= 'binary_crossentropy', optimizer=keras.optimizers.Adam(0.00001, beta_1=0.99, beta_2=0.99), metrics=['accuracy']) #keras.optimizers.Adam(0.000001)
history_1 = model.fit_generator(train_generator,validation_data= val_generator, epochs=50,steps_per_epoch=160,validation_steps=16,callbacks=callbacks_list)

In [ ]:
# mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd


# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_1.history) 


# or save to csv: 
hist_csv_file = '/content/gdrive/My Drive/water_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
model.save("/content/gdrive/My Drive/model_water.h5") 

In [32]:
model_1= keras.models.load_model('/content/gdrive/My Drive/model_water_rescale_only_3.h5')
model_2= keras.models.load_model('/content/gdrive/My Drive/model_water_207_part3.h5')
model_3 = keras.models.load_model('/content/gdrive/My Drive/Copy of model_water_rot_3.h5')
model_4 = keras.models.load_model('/content/gdrive/My Drive/Copy of model_water_shift_3.h5')
model_5 = keras.models.load_model('/content/gdrive/My Drive/model_s_z_3.h5')

In [ ]:
# making predictions 

path_to_image = ''
img = load_img(path_to_image)
x = img_to_array(img)
x = tf.expand_dims(x, axis=0)
x=x/255 # rescale the test image

probs = model.predict(x,steps=100)

preds = (probs > 0.5).astype(int)

# predicted image 
array_to_img(preds[99])